In [1]:
import os
import numpy as np
from netCDF4 import Dataset, num2date
from matplotlib import pyplot as plt, colors
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LinearSegmentedColormap, LogNorm


In [29]:

# Input data directory and variable settings
data_dir = './results/Modis/chla/'
varname = 'chlor_a'
cmin, cmax = -12, 12  # Min and max for color scale

# Define your color list in hex
hex_colors = [
    '#3500a8', '#0800ba', '#003fd6',
    '#00aca9', '#77f800', '#ff8800',
    '#b30000', '#920000', '#880000'
]
rgb_colors = [colors.hex2color(color) for color in hex_colors]
custom_colormap = LinearSegmentedColormap.from_list('custom', rgb_colors)

# Define cropping coordinates
lat_min_crop, lat_max_crop = 34.5, 35.4
lon_min_crop, lon_max_crop = 138.8, 139.9

# Function to process and visualize data
def process_and_visualize(file_path, monthly_means):
    with Dataset(file_path, 'r') as nc:
        data = np.ma.squeeze(nc[varname][:])
        lat = nc['lat'][:]
        lon = nc['lon'][:]
        time = num2date(nc['time'][:], units=nc['time'].units, calendar=nc['time'].calendar)[0]

        # Find indices for cropping
        lat_inds = np.where((lat >= lat_min_crop) & (lat <= lat_max_crop))[0]
        lon_inds = np.where((lon >= lon_min_crop) & (lon <= lon_max_crop))[0]

        if lat_inds.size > 0 and lon_inds.size > 0:
            data_cropped = data[lat_inds, :][:, lon_inds]
        else:
            return  # Skip if no data within bounds
        
        # Compute anomaly
        month = time.month
        anomaly = data_cropped - monthly_means[month]
        
        anomaly = np.ma.filled(anomaly, fill_value=np.nan)  # Replace masked with NaN
        # or
        anomaly = anomaly.data  # Use raw data ignoring the mask

        # Visualization
        plt.figure(figsize=(10, 8))
        m = Basemap(projection='merc', llcrnrlat=lat_min_crop, urcrnrlat=lat_max_crop,
                    llcrnrlon=lon_min_crop, urcrnrlon=lon_max_crop, resolution='i')
        m.drawcoastlines()
        m.drawparallels(np.arange(np.floor(lat_min_crop), np.ceil(lat_max_crop), 0.2), labels=[1,0,0,0])
        m.drawmeridians(np.arange(np.floor(lon_min_crop), np.ceil(lon_max_crop), 0.2), labels=[0,0,0,1])

        lon_cropped, lat_cropped = np.meshgrid(lon[lon_inds], lat[lat_inds])
        x, y = m(lon_cropped, lat_cropped)
        print(x.min(), x.max(), y.min(), y.max())

               # Check and adjust vmin and vmax based on your anomaly data
        vmin = np.min(anomaly)  # For example, set to the minimum anomaly value
        vmax = np.max(anomaly)  # Set to the maximum anomaly value
        # print(anomaly.min(), anomaly.max(),cmin, cmax )
        # print(anomaly.min(), anomaly.max())
        print(np.isnan(anomaly).any())
        print(np.isinf(anomaly).any())
        print(np.ma.is_masked(anomaly))
        print(anomaly)
        # norm = LogNorm(vmin=vmin, vmax=vmax)
        # cs = m.pcolormesh(x, y, anomaly, cmap=custom_colormap, norm=norm)
        # plt.colorbar(cs, orientation='vertical', shrink=0.75)
        plt.title(f'Chlorophyll Anomaly for {time.strftime("%Y-%m")}')
        plt.savefig(f'anomaly_{time.strftime("%Y-%m")}.png', dpi=300)
        plt.close()


In [26]:
# Calculate monthly means
monthly_means = {month: [] for month in range(1, 13)}
for file_name in os.listdir(data_dir):
    if file_name.endswith('.nc'):
        year, month = int(file_name[1:5]), int(file_name[5:7])
        file_path = os.path.join(data_dir, file_name)
        with Dataset(file_path, 'r') as nc:
            data = np.ma.squeeze(nc[varname][:])
            lat = nc['lat'][:]
            lon = nc['lon'][:]
            
            lat_inds = np.where((lat >= lat_min_crop) & (lat <= lat_max_crop))[0]
            lon_inds = np.where((lon >= lon_min_crop) & (lon <= lon_max_crop))[0]
            
            if lat_inds.size > 0 and lon_inds.size > 0:
                data_cropped = data[lat_inds, :][:, lon_inds]
                monthly_means[month].append(data_cropped)
print(len(monthly_means))

12


In [12]:
monthly_means

{1: masked_array(
   data=[[--, --, --, ..., 8.626434326171875, 9.537890625,
          11.94056156703404],
         [--, --, --, ..., 8.908425723805147, 8.70612284342448,
          11.867252056415264],
         [--, --, --, ..., 8.054603576660156, 9.335540771484375,
          5.250819683074951],
         ...,
         [0.29047250747680664, 0.2837490581330799, 0.28463624772571383,
          ..., 0.2545481636410668, 0.26117342994326637,
          0.2526322773524693],
         [0.2806919415791829, 0.2754003433954148, 0.27146743592761813,
          ..., 0.25241942632765996, 0.25822149004255024,
          0.25708153134300593],
         [0.27793116796584355, 0.273897511618478, 0.27506224314371747,
          ..., 0.25322905040922616, 0.2510579427083333,
          0.2478425389244443]],
   mask=[[ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         [ True,  True,  True, ..., False, False, False],
         ...,
         [False, False,

In [30]:
# Calculate mean for each month
for month in monthly_means:
    monthly_means[month] = np.ma.mean(np.ma.stack(monthly_means[month]), axis=0)
# monthly_means
# # Process each file to compute and visualize anomalies
countr = 0
for file_name in os.listdir(data_dir):
    if countr > 0:
        break
    countr += 1
    if file_name.endswith('.nc'):
        file_path = os.path.join(data_dir, file_name)
        process_and_visualize(file_path, monthly_means)

782.1783 121543.05 985.6867970060557 122002.39104384789
True
False
False
